In [ ]:
import pandas as pd
import numpy as np
import requests
import datetime
import time

import firebase_admin
from firebase_admin import credentials, db
from geopy.distance import geodesic

import time
import re
import firebase_admin
from firebase_admin import db
from geopy.distance import geodesic

In [ ]:
cred = credentials.Certificate("creds.json")
firebase_admin.initialize_app(cred, {
    "databaseURL": "https://alert-8c0c7-default-rtdb.asia-southeast1.firebasedatabase.app/"
})

In [ ]:
ambulance_status = db.reference("/ambulance_status").get()
ambulance = db.reference("/ambulence_current_location").get()
hospital = db.reference("/hospitals").get()

In [ ]:
threshold_dist = 0.002

In [ ]:
if not firebase_admin._apps:
    cred = firebase_admin.credentials.Certificate("path/to/serviceAccountKey.json")
    firebase_admin.initialize_app(cred, {"databaseURL": "https://your-database.firebaseio.com"})

def sanitize_key(key):
    return re.sub(r"[^\w]", "_", key)

threshold_dist = 1.0  

def track_ambulances():
    while True:
        ambulance_status = db.reference("/ambulance_status").get()
        if not ambulance_status:
            time.sleep(5)
            continue

        for ambulance_id, hospital_name in ambulance_status.items():
            sanitized_hospital_name = sanitize_key(hospital_name)  
            
            hospital_data = db.reference(f"/hospitals/{sanitized_hospital_name}").get()
            ambulance_data = db.reference(f"/ambulence_current_location/{ambulance_id}").get()

            if hospital_data and ambulance_data:
                hospital_coords = (hospital_data["latitude"], hospital_data["longitude"])
                ambulance_coords = (ambulance_data["latitude"], ambulance_data["longitude"])

                distance = geodesic(hospital_coords, ambulance_coords).km

                if distance < threshold_dist and not ambulance_data.get("isFree", False):
                    print(f"Ambulance {ambulance_id} is near hospital {hospital_name}, distance: {distance:.4f} km")
                    db.reference(f"/ambulence_current_location/{ambulance_id}").update({"isFree": True})

        time.sleep(5)

In [ ]:
track_ambulances()